In [719]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler

## Defining paths and files

In [619]:
data_dir = './Stress-Predict-Dataset-main/Stress-Predict-Dataset-main/Raw_data/'
folders = os.listdir(data_dir)[1:]
temp_file = '/TEMP.csv'
bvp_file = '/BVP.csv'
acc_file = '/ACC.csv'
eda_file = '/EDA.csv'
info_file = '/info.txt'
ibi_file = '/IBI.csv'
hr_file = '/HR.csv'

# chosen_signal_files = [temp_file, bvp_file, eda_file, hr_file, ibi_file]
chosen_signal_files = [temp_file, bvp_file, eda_file, hr_file]
# The accelerometer data is skipped since the hand moves while performing activities and is not a reliable signal
# for stress detection. All other signals are explored

# Exploration of the frequency of ibi signals reveals that the signals are not reliably consistent. Within 5 second
# windows, an average of 6 values is found however it is often 5 or 7 values. While this is alright and can be 
# dealt with, the value often falls to 0 which means no ibi values are recoreded in intervals longer than 5 seconds
# This motivated the decision to drop ibi as a feature as well.

## Initial exploration to fix timestamps

In [556]:
markers = extract_starts_ends()

In [552]:
# This code block was used to help correcting the incorrect timestamps in the excel file
for subject_idx in range(1,36):
    print("Subject: ", subject_idx)
    print(markers[0][subject_idx])
    for file in chosen_signal_files:
        print(get_signal_start(read_data(subject_idx, file)[0][0]))
    print("----------------------------------------")

Subject:  1
2023-04-19 09:22:00
2023-04-19 09:27:41
2023-04-19 09:27:41
2023-04-19 09:27:41
2023-04-19 09:27:51
2023-04-19 09:27:41
----------------------------------------
Subject:  2
2023-04-19 09:48:00
2023-04-19 09:52:54
2023-04-19 09:52:54
2023-04-19 09:52:54
2023-04-19 09:53:04
2023-04-19 09:52:54
----------------------------------------
Subject:  3
2023-04-19 10:50:00
2023-04-19 10:56:12
2023-04-19 10:56:12
2023-04-19 10:56:12
2023-04-19 10:56:22
2023-04-19 10:56:12
----------------------------------------
Subject:  4
2023-04-19 11:25:00
2023-04-19 11:29:49
2023-04-19 11:29:49
2023-04-19 11:29:49
2023-04-19 11:29:59
2023-04-19 11:29:49
----------------------------------------
Subject:  5
2023-04-19 09:05:00
2023-04-19 09:12:05
2023-04-19 09:12:05
2023-04-19 09:12:05
2023-04-19 09:12:15
2023-04-19 09:12:05
----------------------------------------
Subject:  6
2023-04-19 09:41:00
2023-04-19 09:45:00
2023-04-19 09:45:00
2023-04-19 09:45:00
2023-04-19 09:45:10
2023-04-19 09:45:00
---

## Helper functions for data loading and data processing.

In [726]:
def read_tags(subject_idx):
    '''
    This function reads the tags for a given subject and returns the UNIX timestamp of the start of the session 
    followed by the start and end timestamps of each of the three stress inducing sesssions the participant goes
    through. 
    THIS FUNCTION HAS NOT BEEN USED DUE TO INCONSISTENCIES IN TAGS IN RAW DATA!
    '''
    tag_data = pd.read_csv(data_dir + 'S' + (str(subject_idx) if subject_idx > 9 else '0' + str(subject_idx)) + '/tags_' + 'S' + (str(subject_idx) if subject_idx > 9 else '0' + str(subject_idx)) + '.csv', header=None)
    tag_data = tag_data.to_numpy()
    start = tag_data[0]
    stroop_start, stroop_end = tag_data[2], tag_data[3]
    trier_start, trier_end = tag_data[4], tag_data[5]
    hyperv_start, hyperv_end = tag_data[6], tag_data[7]
    return [start, (stroop_start, stroop_end), (trier_start, trier_end), (hyperv_start, hyperv_end)]

def read_data(subject_idx, file):
    '''
    This function reads all signal files.
    Returns (UNIX timestamp for start of signal acquisition, sampling frequency of sensor, acquired data)
    In case of reading Inter-beat interval data, returns (UNIX timestamp for start, arr([seconds since start, duration]))
    '''    
    data = pd.read_csv(data_dir + 'S' + (str(subject_idx) if subject_idx > 9 else '0' + str(subject_idx)) + file, header=None).to_numpy()
    
    if file == '/IBI.csv':
        ibi_start = data[0, 0]
        ibi_data = data[1:]
        return ([ibi_start], 0, ibi_data)
    
    data_start, data_sfreq = data[0], data[1]
    return (data_start, data_sfreq, data[2:])

def convert_to_timestamp(time_str):
    '''This function converts timestamps from excel file into Pandas Timestamp objects.'''
    hours = time_str[:2]
    minutes = time_str[3:5]
    return pd.Timestamp(hours + ':' + minutes)    

def extract_starts_ends(return_from_beg = False):
    '''This function extracts start and end times for stress inducing events from excel file.'''
    timestamps = pd.read_excel(data_dir + 'Time_logs.xlsx')
    
    file_starts, file_ends = timestamps['Start Time'].astype(str)[1:], timestamps['End Time'].astype(str)[1:]
    stroop_starts, stroop_ends = timestamps['Stroop Test'].astype(str)[1:], timestamps['Unnamed: 9'].astype(str)[1:]
    int_starts, int_ends = timestamps['Interview'].astype(str)[1:], timestamps['Unnamed: 13'].astype(str)[1:]
    hypv_starts, hypv_ends = timestamps['Hyperventilation'].astype(str)[1:], timestamps['Unnamed: 17'].astype(str)[1:]
    
    file_starts, file_ends = file_starts.map(convert_to_timestamp), file_ends.map(convert_to_timestamp)
    stroop_starts, stroop_ends = stroop_starts.map(convert_to_timestamp), stroop_ends.map(convert_to_timestamp)
    int_starts, int_ends = int_starts.map(convert_to_timestamp), int_ends.map(convert_to_timestamp)
    hypv_starts, hypv_ends = hypv_starts.map(convert_to_timestamp), hypv_ends.map(convert_to_timestamp)
    
    stress_event_markers = [file_starts, file_ends, stroop_starts, stroop_ends, int_starts, int_ends, hypv_starts, hypv_ends]
    
    if return_from_beg:
        stress_event_markers_from_beg = [file_starts, file_ends]
        for markers in stress_event_markers[1:]:
            stress_event_markers_from_beg.append(markers-file_starts)

        return stress_event_markers_from_beg
    
    else:
        return stress_event_markers

def get_signal_start(unix_start):
    '''This function converts a UNIX timestamp into Pandas Timestamp only preserving the time information.'''
    time = datetime.datetime.fromtimestamp(unix_start)
    return pd.Timestamp(str(time.hour) + ':' + (str(time.minute) if time.minute > 9 else '0' + str(time.minute)) + ':' + str(time.second))

def generate_signal_df(subject_idx, signal_file):
    '''This function generates a Pandas Series for each signal.'''
    data = read_data(subject_idx, signal_file)
    start_time = get_signal_start(data[0][0])
    periods = len(data[2])
    
    if signal_file == temp_file or signal_file == eda_file:
        freq = '250ms'
    elif signal_file == bvp_file:
        freq = '15625us'
    elif signal_file == hr_file:
        freq = '1S'
    else:
        time_idx = []
        for i in range(len(data[2])):
            time_idx.append(start_time + pd.Timedelta(seconds = (data[2][i][0])))
        ibi_df = pd.DataFrame(data[2][:,1], index = time_idx)
        return ibi_df
        
    time_index = pd.date_range(start = start_time, periods=periods, freq=freq)
    df = pd.DataFrame(data[2])
    df.index = time_index
    return df

def gen_label_df(subject_idx, event_markers, epoch_length=5):
    '''This function generates labels for each epoch. Epoch_length is provided in seconds.'''
    event_marks = []
    for i in range(len(event_markers)):
        event_marks.append(event_markers[i][subject_idx])
    freq = str(epoch_length) + 'S'
    time_index = pd.date_range(start = event_marks[0], end = event_marks[1], freq=freq)
    label_df = pd.DataFrame(np.zeros(len(time_index)), index=time_index, columns=['Label'])
    for i in range(1,4):
        idx = label_df.between_time(event_marks[2*i].time(), event_marks[2*i+1].time()).index
        label_df.loc[idx, 'Label'] = 1
    return label_df
    
def gen_epoched_data(subject_idx, markers=extract_starts_ends(),chosen_signal_files=chosen_signal_files, use_scaling=False):
    '''This function uses all methods defined above to generate epoched data with signals for a given subject.'''
    signal_dfs = []
    for file in chosen_signal_files:
        signal_df = generate_signal_df(subject_idx, file)
        if use_scaling:
            scaler = StandardScaler()
            idx = signal_df.index
            signal_df = pd.DataFrame(scaler.fit_transform(signal_df), index=idx)
        signal_dfs.append(signal_df)
    label_df = gen_label_df(subject_idx, markers)
    epoch_timestamps = label_df.index
    X, y = [], []
    for i in range(len(epoch_timestamps) - 1):
        signals = []
        for j in range(len(chosen_signal_files)):
            signals = np.concatenate((signals,
                                  (signal_dfs[j].between_time(epoch_timestamps[i].time(), 
                                                            epoch_timestamps[i+1].time()).to_numpy().reshape(-1))),
                                   axis=0)
        if len(signals) == 369:
            X.append(signals)
            y.append(label_df.loc[epoch_timestamps[i], 'Label'])
    if len(X) == 0:
        print("No epochs returned. Please check to see if the length of signals should still be 369. If not, modify this function. Subject#: ", subject_idx)
    return X, y

## Loading all data without scaling.

In [697]:
x, y = np.empty((0, 369)), np.empty((0))
for i in range(1, 36):
    if i != 17:
        _x, _y = gen_epoched_data(i)
        x = np.concatenate((x, np.array(_x)), axis=0)
        y = np.concatenate((y, np.array(_y)), axis=0)

In [698]:
x, y = np.array(x), np.array(y)

In [699]:
print(x.shape, y.shape)

(22207, 369) (22207,)


## Training a Decision Tree Classifier with 0.25 test split

In [700]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

tree = DecisionTreeClassifier()
x_train, x_test, y_train, y_test = train_test_split(x, y)
tree.fit(x_train, y_train)

DecisionTreeClassifier()

In [701]:
preds = tree.predict(x_test)
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

         0.0       0.85      0.86      0.86      3697
         1.0       0.72      0.69      0.70      1855

    accuracy                           0.81      5552
   macro avg       0.78      0.78      0.78      5552
weighted avg       0.80      0.81      0.81      5552



In [702]:
print(confusion_matrix(preds, y_test))

[[3190  507]
 [ 570 1285]]


In [713]:
tree.score(x_test, y_test)

0.8060158501440923

## Cross-Validation on Decision Tree Classifier

In [765]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree, x, y, cv=4)

In [766]:
print(scores)
print("Average score with cross-validation: ", np.mean(scores))

[0.63004323 0.63814841 0.64355187 0.63502072]
Average score with cross-validation:  0.6366910582095185


## Loading data with scaling

In [727]:
x, y = np.empty((0, 369)), np.empty((0))
for i in range(1, 36):
    if i != 17:
        _x, _y = gen_epoched_data(i, use_scaling=True)
        x = np.concatenate((x, np.array(_x)), axis=0)
        y = np.concatenate((y, np.array(_y)), axis=0)

## Defining MLP and training with 0.25 test split

In [767]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy

model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', metrics='acc', loss=BinaryCrossentropy())

In [768]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [769]:
model.fit(x_train, y_train, epochs=300, batch_size=4096, validation_split=0.15, verbose=False)

In [770]:
model.evaluate(x_test, y_test)

174/174 [==============================] - 0s 728us/step - loss: 2.4937 - acc: 0.6922


[2.4936625957489014, 0.6921830177307129]

## Cross-validation with MLP

In [763]:
from sklearn.model_selection import KFold

n_split=5
scores = []

for train_index,test_index in KFold(n_split).split(x):
    x_train, x_test = x[train_index],x[test_index]
    y_train, y_test = y[train_index],y[test_index]

    model.fit(x_train, y_train,epochs=300, batch_size=4096, verbose=False)

    scores.append(model.evaluate(x_test,y_test)[1])
print(scores)
print("Average score after cross validation: ", np.mean(scores))

139/139 [==============================] - 0s 703us/step - loss: 0.6983 - acc: 0.9124
[0.7861323952674866, 0.739306628704071, 0.8781805634498596, 0.9286196827888489, 0.9124071002006531]
Average score after cross validation:  0.8489292740821839
